As recommeneded by Jeremy, I am writing this blog to deepen my understanding of Deep Learning. Here I am hoping to expand on the material presented in the fastai course and create a full blown MNIST classifier using fastai.

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 11.0MB/s 
     |████████████████████████████████| 194kB 22.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.2MB 39.1MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 12.8MB 222kB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

Quite pissed that I ran into some issues in the import statement itself. After some googling in the fastai forums, I found that apparently the fastai version (v2) needs to be explicitly defined. Somehow the notebooks used in the course circumvents this. Anyway glad to have found the answer within a few minutes of googling. If you have this error just follow this: https://forums.fast.ai/t/modulenotfounderror-no-module-named-fastai-vision-all-on-kaggle-notebook/77008/13?u=nitinkashyap

EDIT: Wrote the above part and then Colab on Google chrome crashed for the umpteenth time. I moved to Brave Browser and the original code works perfectly. Guess there was some issue with Google Chrome.

In [3]:
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

I absolutely love how fastai has managed to improve the readability of python and provide such small elegant functions that used to be a pain for me to create on python jupyter notebooks before. Usually had about three or four lines of code that just set the input and output paths before. Using the untar_data to basically download the data from fastai's AWS S3 on to the working directory and decompress this. this function returns the path of the extracted files. SO SO Elegant!

In [4]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

Another nifty little thing fastai has provided is to access the working directory directly on jupyter books using path.ls(). 

```
path.ls() 
```

I remember toggling back and forth between the terminal and the jupyter notebook while working before. It's the little things like this that let you focus on the deep learning part of the process. 

Big thanks to the fastai team for including so many such features.

In [5]:
path.ls()

(#2) [Path('training'),Path('testing')]

The MNIST dataset follows a common layout for machine learning datasets: separate folders for the training set and the validation set (and/or test set). Let's see what's inside the training set:

In [6]:
(path/'testing').ls()

(#10) [Path('testing/4'),Path('testing/3'),Path('testing/8'),Path('testing/7'),Path('testing/0'),Path('testing/1'),Path('testing/5'),Path('testing/2'),Path('testing/6'),Path('testing/9')]

In [7]:
(path/'training').ls()

(#10) [Path('training/4'),Path('training/3'),Path('training/8'),Path('training/7'),Path('training/0'),Path('training/1'),Path('training/5'),Path('training/2'),Path('training/6'),Path('training/9')]

There are separate folders for inages of each of the 10 digits. Doing this part by brute force, I am sure fastai has some cool functions to make this better. Will explore and update here:

In [8]:
ones = (path/'training'/'1').ls().sorted()
twos = (path/'training'/'2').ls().sorted()
threes = (path/'training'/'3').ls().sorted()
fours = (path/'training'/'4').ls().sorted()
fives = (path/'training'/'5').ls().sorted()
sixes = (path/'training'/'6').ls().sorted()
sevens = (path/'training'/'7').ls().sorted()
eights = (path/'training'/'8').ls().sorted()
nines = (path/'training'/'9').ls().sorted()
zeros = (path/'training'/'0').ls().sorted()

In [9]:
im5 = Image.open(fives[0])
im5

In [10]:
Image??

Here the `Image` class from the *Python Imaging Library* (PIL) is used. This is a Python package for opening, manipulating, and viewing images.

Lets convert these Images to a Tensor so that we can create our model

In [11]:
zero_tensors = [tensor(Image.open(o)) for o in zeros]
one_tensors = [tensor(Image.open(o)) for o in ones]
two_tensors = [tensor(Image.open(o)) for o in twos]
three_tensors = [tensor(Image.open(o)) for o in threes]
four_tensors = [tensor(Image.open(o)) for o in fours]
five_tensors = [tensor(Image.open(o)) for o in fives]
six_tensors = [tensor(Image.open(o)) for o in sixes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]
eight_tensors = [tensor(Image.open(o)) for o in eights]
nine_tensors = [tensor(Image.open(o)) for o in nines]


### Building the Baseline Model: KNN

One of the things emphasized in the fastai course is the need to first create a baseline model. Having an accuracy of over 90% means nothing if a simple arithmetic model has an accuracy of 95%. I have also created many fancy machine learning models only to find that using a simpler model performed far better. A model is useless if it fails to beat a simple benchmark. While it's tempting to rush in and start writing in some fancy code, it might be a good idea to create a simple baseline model to assess the quality of the model's accuracy. Here is the description of a Baseline model from the fastai course:

> jargon: Baseline: A simple model which you are confident should perform reasonably well. It should be very simple to implement, and very easy to test, so that you can then test each of your improved ideas, and make sure they are always better than your baseline. Without starting with a sensible baseline, it is very difficult to know whether your super-fancy models are actually any good. One good approach to creating a baseline is doing what we have done here: think of a simple, easy-to-implement model. Another good approach is to search around to find other people that have solved similar problems to yours, and download and run their code on your dataset. Ideally, try both of these!

As the MNSIT I am trying to do is a bit more complex than the sample MNSIT example in lesson 4 of the course. I am going to try and build a simple KNN Classifier and see how well this performs and hopefully should be able to get a better result using Deep Learning. This will also give me an opportunity of using Tensors to build a KNN



In [12]:
def euc_dist(x1, x2):
  return torch.sqrt(torch.sum((x1-x2)**2))

In [20]:
class KNN:
    def __init__(self, K=3):
        self.K=3
  
    def fit(self, x_train, y_train):
        self.X_train = x_train
        self.y_train = y_train
  
    def predict(self, X_test):
        predictions = []
        for i in range(len(X_test)):
            dist = torch.tensor([euc_dist(X_test[i], x_t) for x_t in self.X_train])
            dist_sorted = torch.argsort(dist)[:self.K]
            neigh_count = {}
            for idx in dist_sorted:
                if self.y_train[idx] in neigh_count:
                    neigh_count[self.y_train[idx]] += 1
                else:
                    neigh_count[self.y_train[idx]] = 1

            sorted_neigh_count = sorted(neigh_count.items(), key=operator.itemgetter(1), reverse=True)
            predictions.append(sorted_neigh_count[0][0]) 
        return predictions       

This script will need a way to create X and Y datasets

In [14]:
flat_zeros = [o.flatten().float()/255 for o in zero_tensors]
flat_ones = [o.flatten().float()/255 for o in one_tensors]
flat_twos = [o.flatten().float()/255 for o in two_tensors]
flat_threes = [o.flatten().float()/255 for o in three_tensors]
flat_fours = [o.flatten().float()/255 for o in four_tensors]
flat_fives = [o.flatten().float()/255 for o in five_tensors]
flat_sixes = [o.flatten().float()/255 for o in six_tensors]
flat_sevens = [o.flatten().float()/255 for o in seven_tensors]
flat_eights = [o.flatten().float()/255 for o in eight_tensors]
flat_nines = [o.flatten().float()/255 for o in nine_tensors]

In [15]:
train_x = torch.cat([torch.stack(flat_zeros), 
                     torch.stack(flat_ones), 
                     torch.stack(flat_twos), 
                     torch.stack(flat_threes), 
                     torch.stack(flat_fours), 
                     torch.stack(flat_fives), 
                     torch.stack(flat_sixes), 
                     torch.stack(flat_sevens), 
                     torch.stack(flat_eights), 
                     torch.stack(flat_nines)])

In [16]:
train_y = tensor([0]*len(zeros)+ 
                 [1]*len(ones)+
                 [2]*len(twos)+
                 [3]*len(threes)+
                 [4]*len(fours)+
                 [5]*len(fives)+
                 [6]*len(sixes)+
                 [7]*len(sevens)+
                 [8]*len(eights)+
                 [9]*len(nines))

Doing the same for the testing data

In [17]:
ones_test = (path/'testing'/'1').ls().sorted()
twos_test = (path/'testing'/'2').ls().sorted()
threes_test = (path/'testing'/'3').ls().sorted()
fours_test = (path/'testing'/'4').ls().sorted()
fives_test = (path/'testing'/'5').ls().sorted()
sixes_test = (path/'testing'/'6').ls().sorted()
sevens_test = (path/'testing'/'7').ls().sorted()
eights_test = (path/'testing'/'8').ls().sorted()
nines_test = (path/'testing'/'9').ls().sorted()
zeros_test = (path/'testing'/'0').ls().sorted()


zero_tensors_test = [tensor(Image.open(o)) for o in zeros_test]
one_tensors_test = [tensor(Image.open(o)) for o in ones_test]
two_tensors_test = [tensor(Image.open(o)) for o in twos_test]
three_tensors_test = [tensor(Image.open(o)) for o in threes_test]
four_tensors_test = [tensor(Image.open(o)) for o in fours_test]
five_tensors_test = [tensor(Image.open(o)) for o in fives_test]
six_tensors_test = [tensor(Image.open(o)) for o in sixes_test]
seven_tensors_test = [tensor(Image.open(o)) for o in sevens_test]
eight_tensors_test = [tensor(Image.open(o)) for o in eights_test]
nine_tensors_test = [tensor(Image.open(o)) for o in nines_test]

flat_zeros_test = [o.flatten().float()/255 for o in zero_tensors_test]
flat_ones_test = [o.flatten().float()/255 for o in one_tensors_test]
flat_twos_test = [o.flatten().float()/255 for o in two_tensors_test]
flat_threes_test = [o.flatten().float()/255 for o in three_tensors_test]
flat_fours_test = [o.flatten().float()/255 for o in four_tensors_test]
flat_fives_test = [o.flatten().float()/255 for o in five_tensors_test]
flat_sixes_test = [o.flatten().float()/255 for o in six_tensors_test]
flat_sevens_test = [o.flatten().float()/255 for o in seven_tensors_test]
flat_eights_test = [o.flatten().float()/255 for o in eight_tensors_test]
flat_nines_test = [o.flatten().float()/255 for o in nine_tensors_test]

test_x = torch.cat([torch.stack(flat_zeros_test), 
                     torch.stack(flat_ones_test), 
                     torch.stack(flat_twos_test), 
                     torch.stack(flat_threes_test), 
                     torch.stack(flat_fours_test), 
                     torch.stack(flat_fives_test), 
                     torch.stack(flat_sixes_test), 
                     torch.stack(flat_sevens_test), 
                     torch.stack(flat_eights_test), 
                     torch.stack(flat_nines_test)])

test_y = tensor([0]*len(zeros_test)+ 
                 [1]*len(ones_test)+
                 [2]*len(twos_test)+
                 [3]*len(threes_test)+
                 [4]*len(fours_test)+
                 [5]*len(fives_test)+
                 [6]*len(sixes_test)+
                 [7]*len(sevens_test)+
                 [8]*len(eights_test)+
                 [9]*len(nines_test))

Running the model to check the results. Got a first hand understanding of how slow the computation is because of the loops

In [ ]:
model = KNN(K=3)
model.fit(train_x, train_y)
pred = model.predict(test_x)

from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred)



1.0

In [26]:
test_y[:100]


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

SyntaxError: ignored

In [ ]:
print(len(three_tensors))
print(three_tensors[0].shape)

In [ ]:
three_tensors[0]

In [ ]:
len(three_tensors[0].flatten())